In [ ]:
#Um estudo desenvolveu um teste online gamificado e um modelo de aprendizado de máquina para detectar dislexia em espanhol, 
#conseguindo identificar mais de 80% dos participantes com dislexia. 
#O método foi testado em mais de 1.300 participantes em um ambiente diferente, 
#alcançando uma taxa de mais de 78% de detecção em crianças a partir de 12 anos. 
#Isso demonstra a viabilidade de triagem de dislexia por meio de aprendizado de máquina.

In [16]:
import pandas as pd
import numpy as np
import csv
from sklearn.model_selection import train_test_split
from collections import defaultdict
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

import xgboost as xgb
from xgboost import XGBClassifier
from sklearn import metrics

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
desktopData= pd.read_csv("../input/dyslexia/Dyt-desktop.csv", index_col=0, na_values=['(NA)'])
tabletData = pd.read_csv("../input/dyslexia/Dyt-tablet.csv", index_col=0, na_values=['(NA)'])

In [2]:
desktopData= pd.read_csv("Dyt-desktop.csv", sep = ';', encoding = 'utf-8')
tabletData = pd.read_csv("Dyt-tablet.csv", sep = ';', encoding = 'utf-8')

In [ ]:
#análise de dados

In [ ]:
desktopData.info()

In [ ]:
desktopData.head()

In [ ]:
tabletData.head()

In [ ]:
desktopData.tail()

In [ ]:
tabletData.tail()

In [ ]:
#comparando os dois dataframes

ColumnDesktopData = list(desktopData.columns.values)
ColumnTabletData = list(tabletData.columns.values)

D_itens_diferentes = [item for item in ColumnDesktopData if item not in ColumnTabletData]
print(D_itens_diferentes)

T_itens_diferentes = [item for item in ColumnTabletData if item not in ColumnDesktopData]
print(T_itens_diferentes)

# os dois dfs possuem as mesmas colunas

In [ ]:
#retorna colunas que são string
colunas_strings = [coluna for coluna in desktopData.columns if desktopData[coluna].apply(lambda x: isinstance(x, str)).all()]

print(colunas_strings)

In [ ]:
#checagem do conteúdo
desktopData.Gender.unique()

In [ ]:
#checagem do conteúdo
desktopData.Nativelang.unique()

In [ ]:
#checagem do conteúdo
desktopData.Otherlang.unique()

In [ ]:
#checagem do conteúdo
desktopData.Dyslexia.unique()

In [ ]:
#verificar se existe valos null
D_null = tabletData.isnull().values.any()
D_null

In [ ]:
# Encontrar as colunas com valores nulos ou NaN
colunas_com_nulls = tabletData.columns[tabletData.isnull().any()]
print(colunas_com_nulls)

In [3]:
#tratamento de dados
#transformar as 4 colunas com string em numerico
def cleanData(data) :
    for col in data.columns.values:
        data[col] = data[col].astype('string')
    #----------
    for col in data.columns.values:
        data[col] = data[col].astype('float',errors = 'ignore')
    #-----------
    data['Gender']=data.Gender.map({'Male': 1, 'Female': 2})
    data['Dyslexia']=data.Dyslexia.map({'No': 0, 'Yes': 1})
    data['Nativelang']=data.Nativelang.map({'No': 0, 'Yes': 1})
    data['Otherlang']=data.Otherlang.map({'No': 0, 'Yes': 1})

In [4]:
cleanData(desktopData)
desktopData.head()

,Gender,Nativelang,Otherlang,Age,Clicks1,Hits1,Misses1,Score1,Accuracy1,Missrate1,...,Score31,Accuracy31,Missrate31,Clicks32,Hits32,Misses32,Score32,Accuracy32,Missrate32,Dyslexia
0,1,0,1,7.0,10.0,10.0,0.0,10.0,1.0,0.0,...,0.0,0.000000,0.00,17.0,2.0,0.0,2.0,0.117647,0.000000,0
1,2,1,1,13.0,12.0,12.0,0.0,12.0,1.0,0.0,...,4.0,0.114286,0.00,26.0,2.0,2.0,2.0,0.076923,0.076923,1
2,2,0,1,7.0,6.0,6.0,0.0,6.0,1.0,0.0,...,4.0,0.114286,0.00,26.0,1.0,3.0,1.0,0.038462,0.115385,0
3,2,0,1,7.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.00,1.0,0.0,0.0,0.0,0.000000,0.000000,0
4,2,0,1,8.0,4.0,4.0,0.0,4.0,1.0,0.0,...,1.0,25.000000,0.05,26.0,2.0,2.0,2.0,0.076923,0.076923,0


In [5]:
cleanData(tabletData)
tabletData.head()

,Gender,Nativelang,Otherlang,Age,Clicks1,Hits1,Misses1,Score1,Accuracy1,Missrate1,...,Score31,Accuracy31,Missrate31,Clicks32,Hits32,Misses32,Score32,Accuracy32,Missrate32,Dyslexia
0,1,1,0,7.0,6.0,6.0,0.0,6.0,1.00,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,2,1,0,7.0,7.0,7.0,0.0,7.0,1.00,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,2,1,0,7.0,6.0,6.0,0.0,6.0,1.00,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,1,1,0,7.0,5.0,5.0,0.0,5.0,1.00,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,1,1,0,7.0,8.0,6.0,2.0,8.0,0.75,0.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [6]:
#tratamento de dados
#tratamento de dados, valor null
stateOfNUll= tabletData.isnull().any()
i = 0
for state in stateOfNUll : 
    if(state):  
        tabletData[stateOfNUll.index[i]].fillna(round(tabletData[stateOfNUll.index[i]].mean() , 4), inplace=True)
    i = i + 1    

tabletData

,Gender,Nativelang,Otherlang,Age,Clicks1,Hits1,Misses1,Score1,Accuracy1,Missrate1,...,Score31,Accuracy31,Missrate31,Clicks32,Hits32,Misses32,Score32,Accuracy32,Missrate32,Dyslexia
0,1,1,0,7.0,6.0,6.0,0.0,6.0,1.00,0.00,...,46.8333,0.386300,0.5439,52.51,2.7851,7.9719,52.51,0.970900,2.225400,0
1,2,1,0,7.0,7.0,7.0,0.0,7.0,1.00,0.00,...,46.8333,0.386300,0.5439,52.51,2.7851,7.9719,52.51,0.970900,2.225400,0
2,2,1,0,7.0,6.0,6.0,0.0,6.0,1.00,0.00,...,46.8333,0.386300,0.5439,52.51,2.7851,7.9719,52.51,0.970900,2.225400,0
3,1,1,0,7.0,5.0,5.0,0.0,5.0,1.00,0.00,...,46.8333,0.386300,0.5439,52.51,2.7851,7.9719,52.51,0.970900,2.225400,0
4,1,1,0,7.0,8.0,6.0,2.0,8.0,0.75,0.25,...,46.8333,0.386300,0.5439,52.51,2.7851,7.9719,52.51,0.970900,2.225400,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1390,1,1,0,17.0,13.0,13.0,0.0,13.0,1.00,0.00,...,35.0000,0.114286,0.0000,26.00,4.0000,0.0000,26.00,0.153846,0.000000,0
1391,2,1,1,17.0,9.0,9.0,0.0,9.0,1.00,0.00,...,35.0000,0.114286,0.0000,26.00,4.0000,0.0000,26.00,0.153846,0.000000,0
1392,1,1,1,17.0,10.0,10.0,0.0,10.0,1.00,0.00,...,35.0000,0.114286,0.0000,27.00,3.0000,2.0000,27.00,0.111111,0.074074,0
1393,2,1,1,17.0,11.0,11.0,0.0,11.0,1.00,0.00,...,35.0000,0.114286,0.0000,26.00,4.0000,0.0000,26.00,0.153846,0.000000,0


In [7]:
#tratamento de dados
#removendo colunas com valor null

cols_with_missing = [col for col in tabletData.columns if tabletData[col].isnull().any()]

# Drop columns desktop data
reduced_desktopData = desktopData.drop(cols_with_missing, axis=1)

# Drop columns tablet data
reduced_tabletData = tabletData.drop(cols_with_missing, axis=1)

In [8]:
commonalityColumns = ['Gender','Nativelang','Otherlang','Age' , 'Dyslexia']
for i in  range(30):
    if((i>=0 and i<12) or (i>=13 and i<17) or i==21 or i==22 or i==29):
        commonalityColumns.append('Clicks'+str(i+1))
        commonalityColumns.append('Hits'+str(i+1))
        commonalityColumns.append('Misses'+str(i+1))
        commonalityColumns.append('Score'+str(i+1))
        commonalityColumns.append('Accuracy'+str(i+1))
        commonalityColumns.append('Missrate'+str(i+1))
    
reduced_desktopData=reduced_desktopData.loc[:,commonalityColumns]
reduced_tabletData=reduced_tabletData.loc[:,commonalityColumns]

In [9]:
#Separando preditor e resposta
y=reduced_desktopData['Dyslexia']
X=reduced_desktopData.loc[:, reduced_desktopData.columns != 'Dyslexia']

In [10]:
#separando treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [17]:
# define data_dmatrix
data_dmatrix = xgb.DMatrix(data=X,label=y)

In [26]:
# declare parameters
params = {
            'objective':'binary:logistic',
            'max_depth': 4,
            'alpha': 10,
            'learning_rate': 1.0,
            'n_estimators':100
        }
            
            
            
# instantiate the classifier 
xgb_clf = XGBClassifier(**params)



# fit the classifier to the training data
xgb_clf.fit(X_train, y_train)

XGBClassifier(alpha=10, base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=1.0, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, ...)

In [27]:
# make predictions on test data
y_pred = xgb_clf.predict(X_test)

In [28]:
# check accuracy score
from sklearn.metrics import accuracy_score

print('XGBoost model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

XGBoost model accuracy score: 0.9019
